In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action="ignore")
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.impute import KNNImputer
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic' # 예시: 맑은 고딕으로 설정 (Windows 환경 시)
plt.rcParams['axes.unicode_minus'] = False

1. **모델학습 및 사용할 피처 PKI 파일 다운 받기**(실행하시기 전에 파일 경로 설정 바꿔주시고  
PKI파일 직접 드리면 가끔 오류 생겨서 코드로 드려요. 실행 안 되면 PKI저장위치 경로 잘 봐주세요)

In [3]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight

# 피처 정의
HE_HP_input_features = ['age','HE_sbp1', 'HE_dbp1',  'HE_wc', 'HE_glu', 'HE_BMI', 'HE_chol', 'HE_LDL_drct', 'HE_HbA1c']
HE_DM_HbA1c_input_features = ['HE_glu','HE_HbA1c', 'HE_wc', 'HE_BMI', 'HE_LDL_drct', 'age'] 
HE_obe_input_features= ['age','HE_wc', 'HE_BMI', 'HE_ht', 'HE_wt', 'HE_alt', 'HE_HDL_st2', 'HE_TG']

# 데이터 로드
#파일 위치 바꿔주세요
data=pd.read_csv("최종_건강검진_데이터셋(가공된).csv")

# HE_HP 데이터 처리
X_hp = data[HE_HP_input_features]
y_hp = data['HE_HP'].astype(int) - 1

# HE_DM_HbA1c 데이터 처리
X_dm = data[HE_DM_HbA1c_input_features]
y_dm = data['HE_DM_HbA1c'].astype(int) - 1

# HE_obe 데이터 처리
X_obe = data[HE_obe_input_features].copy()
y_obe = data['HE_obe'].astype(int) - 1



# 훈련/테스트 분할 (HE_HP)
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X_hp, y_hp, stratify=y_hp, test_size=0.2, random_state=42)

# 훈련/테스트 분할 (HE_DM_HbA1c)
X_train_dm, X_test_dm, y_train_dm, y_test_dm = train_test_split(X_dm, y_dm, stratify=y_dm, test_size=0.2, random_state=42)

# 훈련/테스트 분할 (HE_obe)
X_train_obe, X_test_obe, y_train_obe, y_test_obe = train_test_split(X_obe, y_obe, stratify=y_obe, test_size=0.2, random_state=42)

# 스케일링 (HE_HP)
scaler_hp = StandardScaler()
X_train_hp_scaled = scaler_hp.fit_transform(X_train_hp)
X_test_hp_scaled = scaler_hp.transform(X_test_hp)

# 스케일링 (HE_DM_HbA1c)
scaler_dm = StandardScaler()
X_train_dm_scaled = scaler_dm.fit_transform(X_train_dm)
X_test_dm_scaled = scaler_dm.transform(X_test_dm)

# 스케일링 (HE_obe)
scaler_obe = StandardScaler()
X_train_obe_scaled = scaler_obe.fit_transform(X_train_obe)
X_test_obe_scaled = scaler_obe.transform(X_test_obe)

# SMOTE 적용 (HE_HP)
smote_hp = SMOTE(random_state=42)
X_train_hp_smote, y_train_hp_smote = smote_hp.fit_resample(X_train_hp_scaled, y_train_hp)

# SMOTE 적용 (HE_DM_HbA1c)
smote_dm = SMOTE(random_state=42)
X_train_dm_smote, y_train_dm_smote = smote_dm.fit_resample(X_train_dm_scaled, y_train_dm)

# SMOTE 적용 (HE_obe)
smote_obe = SMOTE(random_state=42)
X_train_obe_smote, y_train_obe_smote = smote_obe.fit_resample(X_train_obe_scaled, y_train_obe)

# 클래스 가중치 계산 (HE_HP)
classes = np.unique(y_train_hp) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_hp)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_hp = dict(zip(classes, weights))

# 클래스 가중치 계산 (HE_DM_HbA1c)
classes = np.unique(y_train_dm) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_dm)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_dm = dict(zip(classes, weights))

# 클래스 가중치 계산 (HE_obe)
classes = np.unique(y_train_obe) # 원본 훈련 데이터의 클래스 레이블
weights = compute_class_weight('balanced', classes=classes, y=y_train_obe)
# 계산된 가중치를 딕셔너리 형태로 변환 {클래스_라벨: 가중치}
class_weights_dict_obe = dict(zip(classes, weights))


# 모델 정의 (HistGradientBoostingClassifier)
base_model_hp = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_hp
)

base_model_dm = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=100,
    max_depth=5,
    min_samples_leaf=20,
    l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_dm
)

base_model_obe = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=50,
    # max_depth=5,
    # min_samples_leaf=20,
    # l2_regularization=1.0,
    random_state=0,
    class_weight=class_weights_dict_obe
)

#모델 훈련
#(HE_HP)
model_hp = CalibratedClassifierCV(base_model_hp, method='sigmoid', cv=5)
model_hp.fit(X_train_hp_smote, y_train_hp_smote)

#(HE_DM_HbA1c)
model_dm = CalibratedClassifierCV(base_model_dm, method='sigmoid', cv=5)
model_dm.fit(X_train_dm_smote, y_train_dm_smote)

#(HE-obe)
model_obe = CalibratedClassifierCV(base_model_obe, method='sigmoid', cv=5)
model_obe.fit(X_train_obe_smote, y_train_obe_smote)
#base_model_obe.fit(X_train_obe_smote, y_train_obe_smote)

#테스트 데이터 평가
#(HE_HP)
y_pred_hp = model_hp.predict(X_test_hp_scaled)
accuracy_hp = accuracy_score(y_test_hp, y_pred_hp) * 100
print("HE_HP Classification Report:")
print(classification_report(y_test_hp, y_pred_hp, target_names=['Normal', 'Pre-hp', 'Pre-high-hp', 'High-hp']))
print(f"HE_HP Accuracy: {accuracy_hp:.1f}%\n") # 정확도 출력
#(HE_DM_HbA1c)
y_pred_dm = model_dm.predict(X_test_dm_scaled)
accuracy_dm = accuracy_score(y_test_dm, y_pred_dm) * 100
print("HE_DM_HbA1c Classification Report:")
print(classification_report(y_test_dm, y_pred_dm, target_names=['Class 0 (Normal)', 'Class 1 (Intermediate)', 'Class 2 (Diabetes)']))
print(f"HE_DM_HbA1c Accuracy: {accuracy_dm:.1f}%\n") # 정확도 출력
#(HE_obe)
y_pred_obe = model_obe.predict(X_test_obe_scaled)
accuracy_obe = accuracy_score(y_test_obe, y_pred_obe) * 100
print("HE_obe Classification Report:")
print(classification_report(y_test_obe, y_pred_obe, target_names=['Low', 'Normal', 'Pre-obese', 'Obese1', 'Obese2', 'Obese3']))
print(f"HE_obe Accuracy: {accuracy_obe:.1f}%\n") # 정확도 출력



# 모델 저장 
#(HE_HP)
calibrated_filename_hp = "For_HE_HP_calibrated_real.pkl"
joblib.dump(model_hp, calibrated_filename_hp)
print(f"HE_HP 모델이 '{calibrated_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
calibrated_filename_dm = "For_HE_DM_HbA1c_calibrated_real.pkl"
joblib.dump(model_dm, calibrated_filename_dm)
print(f"HE_DM_HbA1c 모델이 '{calibrated_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
calibrated_filename_obe = "For_HE_obe_calibrated_real.pkl"
joblib.dump(model_obe, calibrated_filename_obe)
print(f"HE_DM_HbA1c 모델이 '{calibrated_filename_obe}' 파일로 저장되었습니다.")


# 전처리 객체 저장 
#(HE_HP)
scaler_filename_hp = "HE_HP_scaler_real.pkl"
joblib.dump(scaler_hp, scaler_filename_hp)
print(f"HE_HP 스케일러가 '{scaler_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
scaler_filename_dm = "HE_DM_HbA1c_scaler_real.pkl"
joblib.dump(scaler_dm, scaler_filename_dm)
print(f"HE_DM_HbA1c 스케일러가 '{scaler_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
scaler_filename_obe = "HE_obe_scaler_real.pkl"
joblib.dump(scaler_obe, scaler_filename_obe)
print(f"HE_DM_HbA1c 스케일러가 '{scaler_filename_obe}' 파일로 저장되었습니다.")


# 피처 목록 저장 
#(HE_HP)
features_filename_hp = "HE_HP_input_features_list_real.pkl"
joblib.dump(HE_HP_input_features, features_filename_hp)
print(f"HE_HP 피처 목록이 '{features_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
features_filename_dm = "HE_DM_HbA1c_input_features_list_real.pkl"
joblib.dump(HE_DM_HbA1c_input_features, features_filename_dm)
print(f"HE_DM_HbA1c 피처 목록이 '{features_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
features_filename_obe = "HE_obe_input_features_list_real.pkl"
joblib.dump(HE_obe_input_features, features_filename_obe)
print(f"HE_DM_HbA1c 피처 목록이 '{features_filename_obe}' 파일로 저장되었습니다.")


# 훈련 데이터 평균값 저장
# (HE_HP)
train_filename_hp = "train_means_hp_real.pkl"
joblib.dump(X_train_hp.mean(), train_filename_hp)
print(f"HE_HP 훈련 데이터 평균값이 '{train_filename_hp}' 파일로 저장되었습니다.")
#(HE_DM_HbA1c)
train_filename_dm = "train_means_dm_real.pkl"
joblib.dump(X_train_dm.mean(), train_filename_dm)
print(f"HE_DM_HbA1c 훈련 데이터 평균값이 '{train_filename_dm}' 파일로 저장되었습니다.")
#(HE_obe)
train_filename_obe = "train_means_obe_real.pkl"
joblib.dump(X_train_dm.mean(), train_filename_obe)
print(f"HE_obe 훈련 데이터 평균값이 '{train_filename_obe}' 파일로 저장되었습니다.")

# 모델 정확도를 별도 파일로 저장
model_accuracies_to_save = {
    "당뇨병": accuracy_dm,
    "고혈압": accuracy_hp,
    "비만": accuracy_obe
}
joblib.dump(model_accuracies_to_save, 'model_accuracies.pkl')
print(f"모델 정확도가 'model_accuracies.pkl' 파일로 저장되었습니다.")


HE_HP Classification Report:
              precision    recall  f1-score   support

      Normal       0.87      0.84      0.86      1159
      Pre-hp       0.68      0.73      0.71       635
 Pre-high-hp       0.69      0.73      0.71       929
     High-hp       0.66      0.60      0.63       721

    accuracy                           0.74      3444
   macro avg       0.73      0.73      0.73      3444
weighted avg       0.74      0.74      0.74      3444

HE_HP Accuracy: 74.3%

HE_DM_HbA1c Classification Report:
                        precision    recall  f1-score   support

      Class 0 (Normal)       0.99      0.97      0.98      1608
Class 1 (Intermediate)       0.86      0.94      0.90       923
    Class 2 (Diabetes)       0.93      0.88      0.91       913

              accuracy                           0.94      3444
             macro avg       0.93      0.93      0.93      3444
          weighted avg       0.94      0.94      0.94      3444

HE_DM_HbA1c Accuracy: 94.0%

2. **저장된 PKI 불러와 사용자 입력 값을 받을 시 예측 리포트 제공**

In [17]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
import json

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높지만 당뇨병 경계형으로 간주됩니다 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 예측되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압으로 예측되었습니다. 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

user_data = {
    'HE_glu': 125.0,        # 공복 혈당 100~125 → 경계형 당뇨 (유지)
    'HE_HbA1c': 0.0,        # HbA1c 약간 높은 수준 (참고)
    'HE_ht': 170,
    'HE_wt': 63,            # 키 170에 체중 63이면 BMI 약 21.8 → 정상
    'HE_wc': 78,
    'HE_BMI': 21.8,       # BMI 값도 저체중에 맞게 변경
    'HE_sbp1': 135,       # 수축기 혈압 121~139 → 주의 혈압 단계
    'HE_dbp1': 85,        # 이완기 혈압 80~89 → 주의 혈압 단계
    'HE_Upro': 3,
    'HE_chol': 200,
    'HE_HDL_st2': 55,
    'HE_TG': 120,
    'HE_LDL_drct': 110,
    'HE_crea': 0.85,
    'HE_ast': 22,
    'HE_alt': 18,
    'age' : 20,
    #'sex' : 1,
    #'ID' : 100
}

def estimate_hba1c_from_glucose(glu):
    return round((glu + 46.7) / 28.7, 1)

if 'HE_HbA1c' not in user_data or user_data['HE_HbA1c'] is None:
    if 'HE_glu' in user_data:
        estimated_hba1c = estimate_hba1c_from_glucose(user_data['HE_glu'])
        user_data['HE_HbA1c'] = estimated_hba1c
        print(f"당화혈색소(HbA1c) 수치가 없어 공복혈당을 기반으로 {estimated_hba1c}%로 추정했습니다.")
    else:
        print("HbA1c와 공복혈당 모두 없어 평균값으로 대체됩니다.")

# 모델, 스케일러, 피처, 평균값 로드
try:
    model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
    scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
    features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
    train_means_dm = joblib.load('train_means_dm_real.pkl')

    model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
    scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
    features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
    train_means_hp = joblib.load('train_means_hp_real.pkl')

    model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
    scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
    features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
    train_means_obesity = joblib.load('train_means_obe_real.pkl')
except FileNotFoundError as e:
    print(f"Error loading model files: {e}. Please ensure all .pkl files are in the correct directory.")
    exit()

# 모델 정확도 로드
try:
    model_scores = joblib.load('model_accuracies.pkl')
    print("모델 정확도 로드 성공.")
except FileNotFoundError:
    print("경고: 'model_accuracies.pkl' 파일을 찾을 수 없습니다. 모델 정확도가 0으로 처리됩니다.")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }
except Exception as e:
    print(f"모델 정확도 로드 중 오류 발생: {e}")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }

# 사용자 입력 처리 및 예측
user_df_dm, user_scaled_dm = process_user_input(user_data, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)

# 고혈압 예측
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)

# 비만도 예측
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)

def generate_disease_report(user_name, disease_name, result, top_feature_name, user_value):
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1
    }

    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    feature_label_map = {
        "HE_sbp1": "HE_sbp1 (수축기 혈압)",
        "HE_glu": "HE_glu (공복혈당)",
        "HE_BMI": "HE_BMI(BMI)"
    }
    feature_label = feature_label_map.get(top_feature_name, top_feature_name)

    if top_feature_name == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal = "90~120"
        warning = "121~139"
    elif top_feature_name == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal = "70~99"
        warning = "100~125"
    elif top_feature_name == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        elif 25.0 <= user_value <= 29.9:
            status = "비만 1단계"
        elif 30.0 <= user_value <= 34.9:
            status = "비만 2단계"
        else:
            status = "초고도비만"
        normal = "18.5~22.9"
        warning = "23.0~24.9 이상"
    else:
        status = "알 수 없음"
        normal = "-"
        warning = "-"
            
    class_labels_map = {
        "당뇨병": ["정상", "경계형 당뇨", "당뇨병"],
        "고혈압": ["정상 혈압", "주의 혈압 단계", "고혈압 전단계", "고혈압"],
        "비만": ["저체중", "정상 체중", "과체중", "비만", "고도 비만", "초고도 비만"]
    }
    class_labels = class_labels_map.get(disease_name, [f"클래스 {i}" for i in range(len(result["probabilities"]))])

    class_prob_text = ", ".join([
        f"{label} 확률: {float(prob) * 100:.0f}%"
        for label, prob in zip(class_labels, result["probabilities"])
    ])

    current_model_accuracy = model_scores.get(disease_name, None)

    # --- 위험 상태 결정 로직 업데이트 시작 ---
    risk_status_message = ""
    if disease_name == "당뇨병":
        if result['class_name'] == "정상":
            risk_status_message = "정상 상태로 확인됩니다."
        elif result['class_name'] == "경계형 당뇨":
            risk_status_message = "경계형 당뇨 경향이 있습니다." # Changed to '예측됩니다.'
        elif result['class_name'] == "당뇨병":
            risk_status_message = "당뇨병 위험이 있습니다." # Changed to '위험이 있습니다.'
    elif disease_name == "고혈압":
        if result['class_name'] == "정상 혈압":
            risk_status_message = "정상 상태로 확인됩니다."
        elif result['class_name'] == "주의 혈압 단계":
            risk_status_message = "주의 혈압 단계 경향이 있습니다." # Changed to '예측됩니다.'
        elif result['class_name'] == "고혈압 전단계":
            risk_status_message = "고혈압 전단계 경향이 있습니다." # Changed to '예측됩니다.'
        elif result['class_name'] == "고혈압":
            risk_status_message = "고혈압 위험이 있습니다." # Changed to '위험이 있습니다.'
    elif disease_name == "비만":
        if result['class_name'] == "저체중":
            risk_status_message = "저체중 경향이 있습니다." # Changed to '예측됩니다.'
        elif result['class_name'] == "정상 체중":
            risk_status_message = "정상 상태로 확인됩니다.." # Changed to '예측됩니다.'
        elif result['class_name'] == "과체중":
            risk_status_message = "과체중 경향이 있습니다." # Changed to '예측됩니다.'
        elif result['class_name'] in ["비만", "고도비만", "초고도 비만"]: # Grouped for '위험이 있습니다.'
            risk_status_message = f"{result['class_name']} 위험이 있습니다." # Changed to '위험이 있습니다.'
        else:
            risk_status_message = "알 수 없는 비만 관련 상태입니다."
    # --- 위험 상태 결정 로직 업데이트 끝 ---

    print(f"=== {disease_name} 예측 결과 ===")
    print(f"1. {user_name}님, 현재 {disease_name} 관련 상태는 {risk_status_message}")
    print(f"   ({class_prob_text})\n")

    print(f"2. 건강 점수: {health_score:.1f}점 (이 점수는 정상 확률 기준입니다.)\n")

    print(f"3. {feature_label} 수치: {user_value} → 상태: {status}")
    print(f"정상 기준: {normal}, 주의 기준: {warning}")
    print(f"{disease_name}과 가장 높게 연관된 {feature_label.split()[0]}은(는) 높아지면 {disease_name} 위험이 증가합니다.\n")

    return health_score, risk_status_message

# 대표 feature 맵
top_features = {
    "당뇨병": "HE_glu",
    "고혈압": "HE_sbp1",
    "비만": "HE_BMI"
}

# 질병별 결과를 묶기
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

health_scores = []
disease_risk_statuses = {}

for disease_name, result in disease_results.items():
    top_feature = top_features[disease_name]
    user_value = user_data.get(top_feature, None)

    if user_value is not None:
        score, risk_status = generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            top_feature_name=top_feature,
            user_value=user_value
        )
        health_scores.append(score)
        disease_risk_statuses[disease_name] = risk_status
    else:
        print(f"{disease_name}에 대한 {top_feature} 값이 user_data에 없습니다.")

if health_scores:
    total_score = sum(health_scores) / len(health_scores)

summary_list_of_lists = []

feature_name_map = {
    "HE_glu": "공복혈당",
    "HE_sbp1": "수축기 혈압",
    "HE_BMI": "BMI"
}

for disease_name, result in disease_results.items():
    top_feature_code = top_features[disease_name]
    top_feature_name = feature_name_map.get(top_feature_code, top_feature_code)
    user_value = user_data.get(top_feature_code, None)

    if top_feature_code == "HE_sbp1":
        if user_value <= 120:
            status = "정상"
        elif 121 <= user_value <= 139:
            status = "주의"
        else:
            status = "고위험"
        normal_range = "90~120"
    elif top_feature_code == "HE_glu":
        if user_value <= 99:
            status = "정상"
        elif 100 <= user_value <= 125:
            status = "주의"
        else:
            status = "고위험"
        normal_range = "70~99"
    elif top_feature_code == "HE_BMI":
        if user_value < 18.5:
            status = "저체중"
        elif 18.5 <= user_value <= 22.9:
            status = "정상 체중"
        elif 23.0 <= user_value <= 24.9:
            status = "과체중"
        elif 25.0 <= user_value <= 29.9:
            status = "비만"
        elif 30.0 <= user_value <= 34.9:
            status = "고도비만"
        else:
            status = "초고도비만"
        normal_range = "18.5~22.9"
    else:
        status = "알 수 없음"
        normal_range = "-"

    predicted_index = result['predicted_class'] - 1
    predicted_prob = f"{float(result['probabilities'][predicted_index]) * 100:.0f}%"

    value_with_range = (
        f"{str(int(user_value)) if user_value == int(user_value) else str(round(user_value, 1))} "
        f"(정상 기준: {normal_range})"
    )
    
    current_model_accuracy_for_summary = model_scores.get(disease_name, None)
    model_accuracy_str = f"{current_model_accuracy_for_summary:.1f}%" if current_model_accuracy_for_summary is not None else "N/A"

    # --- 최종 리스트에 넣을 risk_status_text 가공 시작 ---
    raw_risk_status_text = disease_risk_statuses.get(disease_name, "N/A")
    # "질병명으로 예측됩니다." 형태에서 "질병명" 부분을 제거
    if raw_risk_status_text.endswith("상태로 확인됩니다."):
        processed_risk_text = "상태로 확인됩니다."
    elif raw_risk_status_text.endswith("경향이 있습니다."):
        processed_risk_text = "경향이 있습니다."
    elif raw_risk_status_text.endswith(" 위험이 있습니다."):
        processed_risk_text = "위험이 있습니다."
    elif raw_risk_status_text.endswith("입니다."): # "정상 체중입니다." 같은 경우
        processed_risk_text = "입니다."
    else:
        processed_risk_text = raw_risk_status_text # 예외 처리
    # --- 최종 리스트에 넣을 risk_status_text 가공 끝 ---

    summary = [
        disease_name,
        result["class_name"],
        predicted_prob,
        model_accuracy_str,
        # 여기에 최종 가공된 processed_risk_text 사용
        processed_risk_text,
        [
            top_feature_name,
            status,
            value_with_range
        ]
    ]
    summary_list_of_lists.append(summary)

print(json.dumps(summary_list_of_lists, indent=2, ensure_ascii=False))

모델 정확도 로드 성공.
=== 당뇨병 예측 결과 ===
1. 홍길동님, 현재 당뇨병 관련 상태는 경계형 당뇨 경향이 있습니다.
   (정상 확률: 0%, 경계형 당뇨 확률: 97%, 당뇨병 확률: 3%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. HE_glu (공복혈당) 수치: 125.0 → 상태: 주의
정상 기준: 70~99, 주의 기준: 100~125
당뇨병과 가장 높게 연관된 HE_glu은(는) 높아지면 당뇨병 위험이 증가합니다.

=== 고혈압 예측 결과 ===
1. 홍길동님, 현재 고혈압 관련 상태는 고혈압 전단계 경향이 있습니다.
   (정상 혈압 확률: 0%, 주의 혈압 단계 확률: 8%, 고혈압 전단계 확률: 83%, 고혈압 확률: 8%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. HE_sbp1 (수축기 혈압) 수치: 135 → 상태: 주의
정상 기준: 90~120, 주의 기준: 121~139
고혈압과 가장 높게 연관된 HE_sbp1은(는) 높아지면 고혈압 위험이 증가합니다.

=== 비만 예측 결과 ===
1. 홍길동님, 현재 비만 관련 상태는 정상 상태로 확인됩니다..
   (저체중 확률: 0%, 정상 체중 확률: 100%, 과체중 확률: 0%, 비만 확률: 0%, 고도 비만 확률: 0%, 초고도 비만 확률: 0%)

2. 건강 점수: 100.0점 (이 점수는 정상 확률 기준입니다.)

3. HE_BMI(BMI) 수치: 21.8 → 상태: 정상
정상 기준: 18.5~22.9, 주의 기준: 23.0~24.9 이상
비만과 가장 높게 연관된 HE_BMI(BMI)은(는) 높아지면 비만 위험이 증가합니다.

[
  [
    "당뇨병",
    "경계형 당뇨",
    "97%",
    "94.0%",
    "경향이 있습니다.",
    [
      "공복혈당",
      "주의",
      "125 (정상 기준: 70~99)"
    ]
  ],
  [
    "고혈압",
   

**질환별 관련 수치 추가**

In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
import json

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높지만 당뇨병 경계형으로 간주됩니다 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 예측되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압으로 예측되었습니다. 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

# 사용자 입력 데이터 (예시)
# HE_HbA1c를 고위험으로 설정하여 테스트
user_data = {
    'HE_glu': 100.0,         # 공복 혈당 (주의)
    'HE_HbA1c': 7.0,         # 당화혈색소 (고위험으로 테스트)
    'HE_ht': 170,
    'HE_wt': 63,
    'HE_wc': 78,            # 허리둘레 (정상으로 테스트)
    'HE_BMI': 21.8,
    'HE_sbp1': 135,
    'HE_dbp1': 85,
    'HE_Upro': 3,
    'HE_chol': 200,
    'HE_HDL_st2': 55,
    'HE_TG': 120,
    'HE_LDL_drct': 110,
    'HE_crea': 0.85,
    'HE_ast': 22,
    'HE_alt': 18,
    'age' : 20,
    'sex' : 1, # 성별 추가: 1은 남성, 2는 여성으로 가정 (일반적인 코딩 방식)
    #'ID' : 100
}

# 공복혈당 기반 당화혈색소 추정 함수 (기존 코드 유지)
def estimate_hba1c_from_glucose(glu):
    return round((glu + 46.7) / 28.7, 1)

# HE_HbA1c 값이 없으면 추정 (기존 코드 유지)
if 'HE_HbA1c' not in user_data or user_data['HE_HbA1c'] is None:
    if 'HE_glu' in user_data and user_data['HE_glu'] is not None and user_data['HE_glu'] > 0:
        estimated_hba1c = estimate_hba1c_from_glucose(user_data['HE_glu'])
        user_data['HE_HbA1c'] = estimated_hba1c
        print(f"당화혈색소(HbA1c) 수치가 없어 공복혈당을 기반으로 {estimated_hba1c}%로 추정했습니다.")
    else:
        print("HbA1c와 공복혈당 모두 없어 평균값으로 대체됩니다.")

# 모델, 스케일러, 피처, 평균값 로드 (기존 코드 유지)
try:
    model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
    scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
    features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
    train_means_dm = joblib.load('train_means_dm_real.pkl')

    model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
    scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
    features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
    train_means_hp = joblib.load('train_means_hp_real.pkl')

    model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
    scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
    features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
    train_means_obesity = joblib.load('train_means_obe_real.pkl')
except FileNotFoundError as e:
    print(f"Error loading model files: {e}. Please ensure all .pkl files are in the correct directory.")
    exit()

# 모델 정확도 로드 (기존 코드 유지)
try:
    model_scores = joblib.load('model_accuracies.pkl')
    print("모델 정확도 로드 성공.")
except FileNotFoundError:
    print("경고: 'model_accuracies.pkl' 파일을 찾을 수 없습니다. 모델 정확도가 0으로 처리됩니다.")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }
except Exception as e:
    print(f"모델 정확도 로드 중 오류 발생: {e}")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }

# --- 입력값 유효성 검사 및 이상치 처리 로직 추가 (기존 코드 유지) ---
def validate_and_process_input(user_data, disease_name):
    validated_data = user_data.copy()
    issue_found = False
    warning_messages = []

    if disease_name == "당뇨병":
        glu = validated_data.get('HE_glu')
        hba1c = validated_data.get('HE_HbA1c')
        if glu is not None:
            if glu < 10 or glu > 600:
                warning_messages.append(f"공복혈당({glu})은(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_glu'] = np.clip(glu, 10, 600)
                issue_found = True
        if hba1c is not None:
            if hba1c < 3 or hba1c > 20:
                warning_messages.append(f"당화혈색소({hba1c})은(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_HbA1c'] = np.clip(hba1c, 3, 20)
                issue_found = True

    elif disease_name == "고혈압":
        sbp = validated_data.get('HE_sbp1')
        dbp = validated_data.get('HE_dbp1')
        if sbp is not None:
            if sbp < 40 or sbp > 300:
                warning_messages.append(f"수축기 혈압({sbp}mmHg)은(는) 비정상적으로 극단적인 값입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_sbp1'] = np.clip(sbp, 40, 300)
                issue_found = True
        if dbp is not None:
            if dbp < 20 or dbp > 180:
                warning_messages.append(f"이완기 혈압({dbp}mmHg)은(는) 비정상적으로 극단적인 값입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_dbp1'] = np.clip(dbp, 20, 180)
                issue_found = True

    elif disease_name == "비만":
        bmi = validated_data.get('HE_BMI')
        ht = validated_data.get('HE_ht')
        wt = validated_data.get('HE_wt')
        wc = validated_data.get('HE_wc')
        if bmi is not None:
            if bmi < 10 or bmi > 60:
                warning_messages.append(f"BMI({bmi})은(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_BMI'] = np.clip(bmi, 10, 60)
                issue_found = True
        if ht is not None and wt is not None:
            calculated_bmi = round(wt / ((ht/100)**2), 1)
            if validated_data.get('HE_BMI') is not None and abs(calculated_bmi - validated_data['HE_BMI']) > 2:
                warning_messages.append(f"입력된 키({ht}cm)와 몸무게({wt}kg)로 계산된 BMI({calculated_bmi})가 입력된 BMI({validated_data.get('HE_BMI')})와 차이가 큽니다. 값을 확인해 주세요.")
                validated_data['HE_BMI'] = calculated_bmi
                issue_found = True
        if wc is not None:
            if wc < 40 or wc > 150:
                warning_messages.append(f"허리둘레({wc}cm)는(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_wc'] = np.clip(wc, 40, 150)
                issue_found = True

    return validated_data, issue_found, warning_messages

# 사용자 입력 처리 및 예측 (기존 코드 유지)
user_data_dm, issue_dm, warnings_dm = validate_and_process_input(user_data, "당뇨병")
user_data_hp, issue_hp, warnings_hp = validate_and_process_input(user_data, "고혈압")
user_data_obesity, issue_obesity, warnings_obesity = validate_and_process_input(user_data, "비만")

user_df_dm, user_scaled_dm = process_user_input(user_data_dm, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data_hp, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data_obesity, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측 (기존 코드 유지)
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)
result_dm['warnings'] = warnings_dm

# 고혈압 예측 (기존 코드 유지)
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)
result_hp['warnings'] = warnings_hp

# 비만도 예측 (기존 코드 유지)
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)
result_obesity['warnings'] = warnings_obesity

# 헬퍼 함수: 값을 포맷하고 정상 기준을 추가 (기존 코드 유지)
def value_with_range_format(val, normal_range_text):
    if val is None:
        return "N/A"
    formatted_val = f"{str(int(val)) if val == int(val) else str(round(val, 1))}"
    return f"{formatted_val} (정상 기준: {normal_range_text})"

# generate_disease_report 함수 수정
def generate_disease_report(user_name, disease_name, result, primary_feature_name, primary_user_value,
                            secondary_feature_name=None, secondary_user_value=None,
                            user_sex=None): # user_sex 매개변수 추가
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1
    }

    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    feature_label_map = {
        "HE_sbp1": "수축기 혈압",
        "HE_dbp1": "이완기 혈압",
        "HE_glu": "공복혈당",
        "HE_HbA1c": "당화혈색소",
        "HE_BMI": "BMI",
        "HE_wc": "허리둘레"
    }
    primary_feature_label = feature_label_map.get(primary_feature_name, primary_feature_name)

    class_labels_map = {
        "당뇨병": ["정상", "경계형 당뇨", "당뇨병"],
        "고혈압": ["정상 혈압", "주의 혈압 단계", "고혈압 전단계", "고혈압"],
        "비만": ["저체중", "정상 체중", "과체중", "비만", "고도 비만", "초고도 비만"]
    }
    
    print(f"=== {disease_name} 예측 결과 ===")
    
    # 경고 메시지 출력
    if result.get('warnings'):
        print(f"⚠️ {disease_name} 관련 입력값 경고:")
        for warning in result['warnings']:
            print(f"   - {warning}")
        print("   (이 경고는 입력된 값이 일반적인 범위를 벗어나 모델 예측에 영향을 줄 수 있음을 나타냅니다.)\n")


    print(f"1. {user_name}님의 현재 {disease_name} 관련 상태는 {result['class_name']} 입니다.")
    print(f"   ({', '.join([f'{label} 확률: {float(prob) * 100:.0f}%' for label, prob in zip(class_labels_map.get(disease_name), result['probabilities'])])})\n")

    print(f"2. 건강 점수: {health_score:.1f}점 (이 점수는 정상 확률 기준입니다.)\n")

    # 핵심 지표에 대한 정보 출력
    if disease_name == "고혈압":
        # 수축기 혈압
        sbp_status = ""
        if primary_user_value is not None:
            if primary_user_value < 120:
                sbp_status = "정상"
            elif 120 <= primary_user_value <= 129:
                sbp_status = "주의혈압"
            elif 130 <= primary_user_value <= 139:
                sbp_status = "위험"
            else: # >= 140
                sbp_status = "고위험"
            
            print(f"3. {primary_feature_label} 수치: {primary_user_value}mmHg → 상태: {sbp_status}")
            print(f"   (정상 <120, 주의혈압압 120-129, 고혈압 전단계 130-139, 고혈압 ≥140)")
        
        # 이완기 혈압
        if secondary_feature_name and secondary_user_value is not None:
            secondary_feature_label = feature_label_map.get(secondary_feature_name, secondary_feature_name)
            dbp_status = ""
            if secondary_user_value < 80:
                dbp_status = "정상"
            elif 80 <= secondary_user_value <= 89:
                dbp_status = "위험"
            else: # >= 90
                dbp_status = "고위험"
            print(f"   {secondary_feature_label} 수치: {secondary_user_value}mmHg → 상태: {dbp_status}")
            print(f"   (정상 <80, 고혈압 1기 80-89, 고혈압 2기 ≥90)\n")
        
        print(f"   {disease_name}과 가장 높게 연관된 수축기/이완기 혈압은(는) 높아지면 고혈압 위험이 증가합니다.\n")

    elif disease_name == "당뇨병":
        # 공복혈당
        if primary_user_value is not None:
            glu_status = ""
            if primary_user_value <= 99:
                glu_status = "정상"
            elif 100 <= primary_user_value <= 125:
                glu_status = "주의"
            else:
                glu_status = "고위험"
            normal_range_glu = "70~99mg/dL"
            warning_range_glu = "100~125mg/dL"
            high_risk_range_glu = "≥126mg/dL" # 고위험 기준 추가

            print(f"3. {primary_feature_label} 수치: {primary_user_value}mg/dL → 상태: {glu_status}")
            print(f"   (정상: {normal_range_glu}, 주의: {warning_range_glu}, 고위험: {high_risk_range_glu})")
        
        # 당화혈색소 (HbA1c) 수정
        if secondary_feature_name and secondary_user_value is not None:
            secondary_feature_label = feature_label_map.get(secondary_feature_name, secondary_feature_name)
            hba1c_status = ""
            if secondary_user_value < 5.7:
                hba1c_status = "정상"
            elif 5.7 <= secondary_user_value <= 6.4:
                hba1c_status = "주의"
            else: # 6.5 이상이면 고위험
                hba1c_status = "고위험"
            normal_range_hba1c = "<5.7%"
            warning_range_hba1c = "5.7%~6.4%"
            high_risk_range_hba1c = "≥6.5%" # 고위험 기준 추가
            
            print(f"   {secondary_feature_label} 수치: {secondary_user_value}% → 상태: {hba1c_status}")
            print(f"   (정상: {normal_range_hba1c}, 주의: {warning_range_hba1c}, 고위험: {high_risk_range_hba1c})\n")
        
        print(f"   {disease_name}과 가장 높게 연관된 공복혈당과 당화혈색소는(는) 높아지면 {disease_name} 위험이 증가합니다.\n")


    elif disease_name == "비만":
        # BMI
        if primary_user_value is not None:
            bmi_status = ""
            if primary_user_value < 18.5:
                bmi_status = "저체중"
            elif 18.5 <= primary_user_value <= 22.9:
                bmi_status = "정상 체중"
            elif 23.0 <= primary_user_value <= 24.9:
                bmi_status = "과체중"
            elif 25.0 <= primary_user_value <= 29.9:
                bmi_status = "비만 1단계"
            elif 30.0 <= primary_user_value <= 34.9:
                bmi_status = "비만 2단계"
            else:
                bmi_status = "초고도비만"
            normal_range_bmi = "18.5~22.9"
            warning_range_bmi = "23.0~24.9 이상" # 여기서는 '이상'이 과체중을 의미
            
            print(f"3. {primary_feature_label} 수치: {primary_user_value} → 상태: {bmi_status}")
            print(f"   (정상: {normal_range_bmi}, 과체중: {warning_range_bmi})")
            
        # 허리둘레 (HE_wc) 수정
        if secondary_feature_name and secondary_user_value is not None:
            secondary_feature_label = feature_label_map.get(secondary_feature_name, secondary_feature_name)
            wc_status = ""
            wc_normal_male = 90
            wc_normal_female = 85
            
            # 성별에 따른 허리둘레 기준 적용
            if user_sex == 1: # 남성
                if secondary_user_value < wc_normal_male:
                    wc_status = "정상"
                else:
                    wc_status = "위험"
                normal_range_wc_text = f"< {wc_normal_male}cm"
                
            elif user_sex == 2: # 여성
                if secondary_user_value < wc_normal_female:
                    wc_status = "정상"
                else:
                    wc_status = "위험"
                normal_range_wc_text = f"< {wc_normal_female}cm"
            else: # 성별 정보가 없거나 불분명할 경우, 일반적인 기준을 명시
                if secondary_user_value <= 80: # 일반적인 참고치 (남녀공통으로 대략 80cm를 기준으로 함)
                    wc_status = "정상"
                elif 81 <= secondary_user_value <= 89:
                    wc_status = "주의" # 임의의 주의 구간
                else:
                    wc_status = "위험"
                normal_range_wc_text = f"남성 <{wc_normal_male}cm, 여성 <{wc_normal_female}cm" # 일반적인 참고 기준

            print(f"   {secondary_feature_label} 수치: {secondary_user_value}cm → 상태: {wc_status}")
            if user_sex == 1:
                print(f"   (정상 기준: 남성 {normal_range_wc_text})")
            elif user_sex == 2:
                print(f"   (정상 기준: 여성 {normal_range_wc_text})")
            else:
                print(f"   (정상 기준: {normal_range_wc_text})")
            print("") # 줄바꿈

        print(f"   {disease_name}과 가장 높게 연관된 {primary_feature_label}과 {secondary_feature_label}은(는) 높아지면 {disease_name} 위험이 증가합니다.\n")


    return health_score, result['class_name']

# 대표 feature 맵 (주요 지표와 보조 지표)
top_features_map = {
    "당뇨병": {"primary": "HE_glu", "secondary": "HE_HbA1c"},
    "고혈압": {"primary": "HE_sbp1", "secondary": "HE_dbp1"},
    "비만": {"primary": "HE_BMI", "secondary": "HE_wc"}
}

# 질병별 결과를 묶기 (기존 코드 유지)
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

health_scores = []
disease_risk_statuses = {}

for disease_name, result in disease_results.items():
    primary_feature = top_features_map[disease_name]["primary"]
    secondary_feature = top_features_map[disease_name].get("secondary")

    current_validated_user_data = None
    if disease_name == "당뇨병":
        current_validated_user_data = user_data_dm
    elif disease_name == "고혈압":
        current_validated_user_data = user_data_hp
    elif disease_name == "비만":
        current_validated_user_data = user_data_obesity
    
    # 성별 정보 전달
    user_sex_val = current_validated_user_data.get('sex')


    primary_user_value = current_validated_user_data.get(primary_feature)
    secondary_user_value = current_validated_user_data.get(secondary_feature) if secondary_feature else None

    if primary_user_value is not None:
        score, risk_status = generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            primary_feature_name=primary_feature,
            primary_user_value=primary_user_value,
            secondary_feature_name=secondary_feature,
            secondary_user_value=secondary_user_value,
            user_sex=user_sex_val # 성별 정보 전달
        )
        health_scores.append(score)
        disease_risk_statuses[disease_name] = risk_status
    else:
        print(f"{disease_name}에 대한 {primary_feature} 값이 user_data에 없습니다.")

# 총점 출력은 딱 한 번, 루프 끝나고! (기존 코드 유지)
if health_scores:
    total_score = sum(health_scores) / len(health_scores)
    print("---")
    print(f"### 총 개인 건강 점수: {total_score:.1f}점")
    print("(당뇨병, 고혈압, 비만에 대한 정상상태 점수를 평균한 값입니다.)\n")

# summary_list_of_lists 생성 부분 수정
summary_list_of_lists = []

feature_name_map = {
    "HE_glu": "공복혈당",
    "HE_HbA1c": "당화혈색소",
    "HE_sbp1": "수축기 혈압",
    "HE_dbp1": "이완기 혈압",
    "HE_BMI": "BMI",
    "HE_wc": "허리둘레"
}

for disease_name, result in disease_results.items():
    primary_feature_code = top_features_map[disease_name]["primary"]
    secondary_feature_code = top_features_map[disease_name].get("secondary")

    current_validated_user_data = None
    if disease_name == "당뇨병":
        current_validated_user_data = user_data_dm
    elif disease_name == "고혈압":
        current_validated_user_data = user_data_hp
    elif disease_name == "비만":
        current_validated_user_data = user_data_obesity
    
    user_sex_val = current_validated_user_data.get('sex') # 성별 정보 가져오기

    user_value_primary = current_validated_user_data.get(primary_feature_code)
    user_value_secondary = current_validated_user_data.get(secondary_feature_code) if secondary_feature_code else None

    # 각 질병별로 primary/secondary feature 상태 및 값 포맷팅
    # 고혈압
    if disease_name == "고혈압":
        sbp_status = ""
        if user_value_primary is not None:
            if user_value_primary < 120: sbp_status = "정상"
            elif 120 <= user_value_primary <= 129: sbp_status = "주의"
            elif 130 <= user_value_primary <= 139: sbp_status = "위험"
            else: sbp_status = "고위험"
        normal_range_sbp = "<120mmHg"

        dbp_status = ""
        if user_value_secondary is not None:
            if user_value_secondary < 80: dbp_status = "정상"
            elif 80 <= user_value_secondary <= 89: dbp_status = "위험"
            else: dbp_status = "고위험"
        normal_range_dbp = "<80mmHg"
        
        primary_feature_display_text = f"수축기: {sbp_status} ({user_value_primary}mmHg), 이완기: {dbp_status} ({user_value_secondary}mmHg)"
        value_with_range_primary_for_summary = f"수축기: {value_with_range_format(user_value_primary, normal_range_sbp)}, 이완기: {value_with_range_format(user_value_secondary, normal_range_dbp)}"

    # 당뇨병
    elif disease_name == "당뇨병":
        glu_status = ""
        if user_value_primary is not None:
            if user_value_primary <= 99: glu_status = "정상"
            elif 100 <= user_value_primary <= 125: glu_status = "주의"
            else: glu_status = "고위험"
        normal_range_glu = "70~99mg/dL"
        
        hba1c_status = ""
        if user_value_secondary is not None:
            if user_value_secondary < 5.7: hba1c_status = "정상"
            elif 5.7 <= user_value_secondary <= 6.4: hba1c_status = "주의"
            else: hba1c_status = "고위험" # 고위험 추가
        normal_range_hba1c = "<5.7%"

        primary_feature_display_text = f"공복혈당: {glu_status} ({user_value_primary}mg/dL), 당화혈색소: {hba1c_status} ({user_value_secondary}%)"
        value_with_range_primary_for_summary = f"공복혈당: {value_with_range_format(user_value_primary, normal_range_glu)}, 당화혈색소: {value_with_range_format(user_value_secondary, normal_range_hba1c)}"
    
    # 비만
    elif disease_name == "비만":
        bmi_status = ""
        if user_value_primary is not None:
            if user_value_primary < 18.5: bmi_status = "저체중"
            elif 18.5 <= user_value_primary <= 22.9: bmi_status = "정상 체중"
            elif 23.0 <= user_value_primary <= 24.9: bmi_status = "과체중"
            elif 25.0 <= user_value_primary <= 29.9: bmi_status = "비만 1단계"
            elif 30.0 <= user_value_primary <= 34.9: bmi_status = "비만 2단계"
            else: bmi_status = "초고도비만"
        normal_range_bmi = "18.5~22.9"

        wc_status = ""
        normal_range_wc_text_summary = ""
        if user_value_secondary is not None:
            if user_sex_val == 1: # 남성
                if user_value_secondary < 90:
                    wc_status = "정상"
                else:
                    wc_status = "위험"
                normal_range_wc_text_summary = "<90cm (남성 기준)"
            elif user_sex_val == 2: # 여성
                if user_value_secondary < 85:
                    wc_status = "정상"
                else:
                    wc_status = "위험"
                normal_range_wc_text_summary = "<85cm (여성 기준)"
            else: # 성별 정보 없을 때
                if user_value_secondary <= 80:
                    wc_status = "정상"
                elif 81 <= user_value_secondary <= 89:
                    wc_status = "주의"
                else:
                    wc_status = "위험"
                normal_range_wc_text_summary = "남성 <90cm, 여성 <85cm (일반적인 기준)"


        primary_feature_display_text = f"BMI: {bmi_status} ({user_value_primary}), 허리둘레: {wc_status} ({user_value_secondary}cm)"
        value_with_range_primary_for_summary = f"BMI: {value_with_range_format(user_value_primary, normal_range_bmi)}, 허리둘레: {value_with_range_format(user_value_secondary, normal_range_wc_text_summary)}"
    else:
        # 기타 질병에 대한 기본 처리
        primary_feature_display_text = f"{feature_name_map.get(primary_feature_code, primary_feature_code)}: N/A ({user_value_primary})"
        value_with_range_primary_for_summary = "N/A"
    
    predicted_index = result['predicted_class'] - 1
    predicted_prob = f"{float(result['probabilities'][predicted_index]) * 100:.0f}%"

    current_model_accuracy_for_summary = model_scores.get(disease_name, None)
    model_accuracy_str = f"{current_model_accuracy_for_summary:.1f}%" if current_model_accuracy_for_summary is not None else "N/A"

    processed_risk_text = disease_risk_statuses.get(disease_name, "N/A")

    summary = [
        disease_name,
        result["class_name"],
        processed_risk_text,
        predicted_prob,
        model_accuracy_str,
        [
            primary_feature_display_text,
            value_with_range_primary_for_summary
        ]
    ]
    summary_list_of_lists.append(summary)

print(json.dumps(summary_list_of_lists, indent=2, ensure_ascii=False))

모델 정확도 로드 성공.
=== 당뇨병 예측 결과 ===
1. 홍길동님의 현재 당뇨병 관련 상태는 당뇨병 입니다.
   (정상 확률: 0%, 경계형 당뇨 확률: 46%, 당뇨병 확률: 54%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. 공복혈당 수치: 100.0mg/dL → 상태: 주의
   (정상: 70~99mg/dL, 주의: 100~125mg/dL, 고위험: ≥126mg/dL)
   당화혈색소 수치: 7.0% → 상태: 고위험
   (정상: <5.7%, 주의: 5.7%~6.4%, 고위험: ≥6.5%)

   당뇨병과 가장 높게 연관된 공복혈당과 당화혈색소는(는) 높아지면 당뇨병 위험이 증가합니다.

=== 고혈압 예측 결과 ===
1. 홍길동님의 현재 고혈압 관련 상태는 고혈압 전단계 입니다.
   (정상 혈압 확률: 0%, 주의 혈압 단계 확률: 40%, 고혈압 전단계 확률: 59%, 고혈압 확률: 2%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. 수축기 혈압 수치: 135mmHg → 상태: 위험
   (정상 <120, 주의혈압압 120-129, 고혈압 전단계 130-139, 고혈압 ≥140)
   이완기 혈압 수치: 85mmHg → 상태: 위험
   (정상 <80, 고혈압 1기 80-89, 고혈압 2기 ≥90)

   고혈압과 가장 높게 연관된 수축기/이완기 혈압은(는) 높아지면 고혈압 위험이 증가합니다.

=== 비만 예측 결과 ===
1. 홍길동님의 현재 비만 관련 상태는 정상 체중 입니다.
   (저체중 확률: 0%, 정상 체중 확률: 100%, 과체중 확률: 0%, 비만 확률: 0%, 고도 비만 확률: 0%, 초고도 비만 확률: 0%)

2. 건강 점수: 100.0점 (이 점수는 정상 확률 기준입니다.)

3. BMI 수치: 21.8 → 상태: 정상 체중
   (정상: 18.5~22.9, 과체중: 23.0~24.9 이상)
   허리둘레 수치: 78cm → 상태: 정상
   (정상 기준:

In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
import json

# 사용자 입력 처리 함수
def process_user_input(user_data, features, scaler, train_means):
    user_df = pd.DataFrame([user_data]).reindex(columns=features)
    user_df = user_df.fillna(train_means)  # 누락된 피처를 평균값으로 채움
    user_scaled = scaler.transform(user_df)
    return user_df, user_scaled

# 예측 함수
def predict_disease_probabilities(user_scaled, model, stages, class_explanations):
    probs = model.predict_proba(user_scaled)[0]
    stage_index = np.argmax(probs)
    stage = stages.get(stage_index, "Unknown")
    explanation = class_explanations.get(stage_index, "No explanation available.")
    class_probs_formatted = [f"{probs[i]:.2f}" for i in range(len(probs))]
    return {
        "probabilities": class_probs_formatted,
        "predicted_class": stage_index + 1,
        "class_name": stage,
        "explanation": explanation
    }

# 각 클래스의 설명
diabetes_class_explanations = {
    0: "혈당 수준이 정상입니다. 당뇨병 위험이 낮습니다.",
    1: "혈당이 정상보다 높지만 당뇨병 경계형으로 간주됩니다 생활습관 개선이 필요합니다.",
    2: "당뇨병으로 예측되었습니다. 전문가와 상담이 필요합니다."
}

hypertension_class_explanations = {
    0: "혈압이 정상 범위에 있습니다.",
    1: "혈압이 정상보다 높지만 고혈압 전단계로 간주됩니다. 건강 관리가 필요합니다.",
    2: "고혈압 전단계로, 생활습관 개선과 치료가 필요할 수 있습니다.",
    3: "고혈압으로 예측되었습니다. 전문가의 진단과 치료가 필요합니다."
}

obesity_class_explanations = {
    0: "저체중입니다. 체중 증가가 필요할 수 있습니다.",
    1: "정상 체중입니다. 유지 관리가 필요합니다.",
    2: "과체중입니다. 체중 감량을 고려할 수 있습니다.",
    3: "비만입니다. 체중 감량과 건강 관리가 필요합니다.",
    4: "고도 비만입니다. 전문가의 진단과 관리가 필요합니다.",
    5: "초고도 비만입니다. 전문가의 도움을 받아야 합니다."
}

# 사용자 입력 데이터 (예시)
user_data = {
    'HE_glu': 100.0,         # 공복 혈당 (주의)
    'HE_HbA1c': 7.0,         # 당화혈색소 (고위험으로 테스트)
    'HE_ht': 170,
    'HE_wt': 63,
    'HE_wc': 78,            # 허리둘레 (정상으로 테스트)
    'HE_BMI': 21.8,
    'HE_sbp1': 135,
    'HE_dbp1': 85,
    'HE_Upro': 3,
    'HE_chol': 200,
    'HE_HDL_st2': 55,
    'HE_TG': 120,
    'HE_LDL_drct': 110,
    'HE_crea': 0.85,
    'HE_ast': 22,
    'HE_alt': 18,
    'age' : 20,
    'sex' : 1, # 성별 추가: 1은 남성, 2는 여성으로 가정 (일반적인 코딩 방식)
}

# 공복혈당 기반 당화혈색소 추정 함수 (기존 코드 유지)
def estimate_hba1c_from_glucose(glu):
    return round((glu + 46.7) / 28.7, 1)

# HE_HbA1c 값이 없으면 추정 (기존 코드 유지)
if 'HE_HbA1c' not in user_data or user_data['HE_HbA1c'] is None:
    if 'HE_glu' in user_data and user_data['HE_glu'] is not None and user_data['HE_glu'] > 0:
        estimated_hba1c = estimate_hba1c_from_glucose(user_data['HE_glu'])
        user_data['HE_HbA1c'] = estimated_hba1c
        print(f"당화혈색소(HbA1c) 수치가 없어 공복혈당을 기반으로 {estimated_hba1c}%로 추정했습니다.")
    else:
        print("HbA1c와 공복혈당 모두 없어 평균값으로 대체됩니다.")

# 모델, 스케일러, 피처, 평균값 로드 (기존 코드 유지)
try:
    model_dm = joblib.load('For_HE_DM_HbA1c_calibrated_real.pkl')
    scaler_dm = joblib.load('HE_DM_HbA1c_scaler_real.pkl')
    features_dm = joblib.load('HE_DM_HbA1c_input_features_list_real.pkl')
    train_means_dm = joblib.load('train_means_dm_real.pkl')

    model_hp = joblib.load('For_HE_HP_calibrated_real.pkl')
    scaler_hp = joblib.load('HE_HP_scaler_real.pkl')
    features_hp = joblib.load('HE_HP_input_features_list_real.pkl')
    train_means_hp = joblib.load('train_means_hp_real.pkl')

    model_obesity = joblib.load('For_HE_obe_calibrated_real.pkl')
    scaler_obesity = joblib.load('HE_obe_scaler_real.pkl')
    features_obesity = joblib.load('HE_obe_input_features_list_real.pkl')
    train_means_obesity = joblib.load('train_means_obe_real.pkl')
except FileNotFoundError as e:
    print(f"Error loading model files: {e}. Please ensure all .pkl files are in the correct directory.")
    exit()

# 모델 정확도 로드 (기존 코드 유지)
try:
    model_scores = joblib.load('model_accuracies.pkl')
    print("모델 정확도 로드 성공.")
except FileNotFoundError:
    print("경고: 'model_accuracies.pkl' 파일을 찾을 수 없습니다. 모델 정확도가 0으로 처리됩니다.")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }
except Exception as e:
    print(f"모델 정확도 로드 중 오류 발생: {e}")
    model_scores = {
        "당뇨병": 0.0,
        "고혈압": 0.0,
        "비만": 0.0
    }

# --- 입력값 유효성 검사 및 이상치 처리 로직 추가 (기존 코드 유지) ---
def validate_and_process_input(user_data, disease_name):
    validated_data = user_data.copy()
    issue_found = False
    warning_messages = []

    if disease_name == "당뇨병":
        glu = validated_data.get('HE_glu')
        hba1c = validated_data.get('HE_HbA1c')
        if glu is not None:
            if glu < 10 or glu > 600:
                warning_messages.append(f"공복혈당({glu})은(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_glu'] = np.clip(glu, 10, 600)
                issue_found = True
        if hba1c is not None:
            if hba1c < 3 or hba1c > 20:
                warning_messages.append(f"당화혈색소({hba1c})은(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_HbA1c'] = np.clip(hba1c, 3, 20)
                issue_found = True

    elif disease_name == "고혈압":
        sbp = validated_data.get('HE_sbp1')
        dbp = validated_data.get('HE_dbp1')
        if sbp is not None:
            if sbp < 40 or sbp > 300:
                warning_messages.append(f"수축기 혈압({sbp}mmHg)은(는) 비정상적으로 극단적인 값입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_sbp1'] = np.clip(sbp, 40, 300)
                issue_found = True
        if dbp is not None:
            if dbp < 20 or dbp > 180:
                warning_messages.append(f"이완기 혈압({dbp}mmHg)은(는) 비정상적으로 극단적인 값입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_dbp1'] = np.clip(dbp, 20, 180)
                issue_found = True

    elif disease_name == "비만":
        bmi = validated_data.get('HE_BMI')
        ht = validated_data.get('HE_ht')
        wt = validated_data.get('HE_wt')
        wc = validated_data.get('HE_wc')
        if bmi is not None:
            if bmi < 10 or bmi > 60:
                warning_messages.append(f"BMI({bmi})은(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_BMI'] = np.clip(bmi, 10, 60)
                issue_found = True
        if ht is not None and wt is not None:
            calculated_bmi = round(wt / ((ht/100)**2), 1)
            if validated_data.get('HE_BMI') is not None and abs(calculated_bmi - validated_data['HE_BMI']) > 2:
                warning_messages.append(f"입력된 키({ht}cm)와 몸무게({wt}kg)로 계산된 BMI({calculated_bmi})가 입력된 BMI({validated_data.get('HE_BMI')})와 차이가 큽니다. 값을 확인해 주세요.")
                validated_data['HE_BMI'] = calculated_bmi
                issue_found = True
        if wc is not None:
            if wc < 40 or wc > 150:
                warning_messages.append(f"허리둘레({wc}cm)는(는) 비정상적인 범위입니다. 실제 값인지 확인해 주세요.")
                validated_data['HE_wc'] = np.clip(wc, 40, 150)
                issue_found = True

    return validated_data, issue_found, warning_messages

# 사용자 입력 처리 및 예측
user_data_dm, issue_dm, warnings_dm = validate_and_process_input(user_data, "당뇨병")
user_data_hp, issue_hp, warnings_hp = validate_and_process_input(user_data, "고혈압")
user_data_obesity, issue_obesity, warnings_obesity = validate_and_process_input(user_data, "비만")


user_df_dm, user_scaled_dm = process_user_input(user_data_dm, features_dm, scaler_dm, train_means_dm)
user_df_hp, user_scaled_hp = process_user_input(user_data_hp, features_hp, scaler_hp, train_means_hp)
user_df_obesity, user_scaled_obesity = process_user_input(user_data_obesity, features_obesity, scaler_obesity, train_means_obesity)

# 당뇨병 예측
diabetes_stages = {0: "정상", 1: "경계형 당뇨", 2: "당뇨병"}
result_dm = predict_disease_probabilities(user_scaled_dm, model_dm, diabetes_stages, diabetes_class_explanations)
result_dm['warnings'] = warnings_dm

# 고혈압 예측
hypertension_stages = {0: "정상 혈압", 1: "주의 혈압 단계", 2: "고혈압 전단계", 3: "고혈압"}
result_hp = predict_disease_probabilities(user_scaled_hp, model_hp, hypertension_stages, hypertension_class_explanations)
result_hp['warnings'] = warnings_hp

# 비만도 예측
obesity_stages = {0: "저체중", 1: "정상 체중", 2: "과체중", 3: "비만", 4: "고도비만", 5: "초고도 비만"}
result_obesity = predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages, obesity_class_explanations)
result_obesity['warnings'] = warnings_obesity

# 헬퍼 함수: 값을 포맷하고 정상 기준을 추가 (기존 코드 유지)
def value_with_range_format(val, normal_range_text):
    if val is None:
        return "N/A"
    formatted_val = f"{str(int(val)) if val == int(val) else str(round(val, 1))}"
    return f"{formatted_val} (정상 기준: {normal_range_text})"

# generate_disease_report 함수는 콘솔 출력을 위한 것이므로, JSON 출력 로직과 분리하여 유지
def generate_disease_report(user_name, disease_name, result, primary_feature_name, primary_user_value,
                            secondary_feature_name=None, secondary_user_value=None,
                            user_sex=None):
    normal_class_index = {
        "당뇨병": 0,
        "고혈압": 0,
        "비만": 1
    }

    normal_index = normal_class_index.get(disease_name, 0)
    prob_percent = float(result['probabilities'][normal_index]) * 100
    health_score = round(prob_percent, 1)

    predicted_index = result['predicted_class'] - 1
    predicted_prob = float(result['probabilities'][predicted_index]) * 100

    feature_label_map = {
        "HE_sbp1": "수축기 혈압",
        "HE_dbp1": "이완기 혈압",
        "HE_glu": "공복혈당",
        "HE_HbA1c": "당화혈색소",
        "HE_BMI": "BMI",
        "HE_wc": "허리둘레"
    }
    primary_feature_label = feature_label_map.get(primary_feature_name, primary_feature_name)

    class_labels_map = {
        "당뇨병": ["정상", "경계형 당뇨", "당뇨병"],
        "고혈압": ["정상 혈압", "주의 혈압 단계", "고혈압 전단계", "고혈압"],
        "비만": ["저체중", "정상 체중", "과체중", "비만", "고도 비만", "초고도 비만"]
    }
    
    print(f"=== {disease_name} 예측 결과 ===")
    
    if result.get('warnings'):
        print(f"⚠️ {disease_name} 관련 입력값 경고:")
        for warning in result['warnings']:
            print(f"   - {warning}")
        print("   (이 경고는 입력된 값이 일반적인 범위를 벗어나 모델 예측에 영향을 줄 수 있음을 나타냅니다.)\n")

    print(f"1. {user_name}님의 현재 {disease_name} 관련 상태는 {result['class_name']} 입니다.")
    print(f"   ({', '.join([f'{label} 확률: {float(prob) * 100:.0f}%' for label, prob in zip(class_labels_map.get(disease_name), result['probabilities'])])})\n")

    print(f"2. 건강 점수: {health_score:.1f}점 (이 점수는 정상 확률 기준입니다.)\n")

    # 핵심 지표에 대한 정보 출력 (콘솔 출력용)
    if disease_name == "고혈압":
        sbp_status = ""
        if primary_user_value is not None:
            if primary_user_value < 120: sbp_status = "정상"
            elif 120 <= primary_user_value <= 129: sbp_status = "주의혈압"
            elif 130 <= primary_user_value <= 139: sbp_status = "위험"
            else: sbp_status = "고위험"
            print(f"3. {primary_feature_label} 수치: {primary_user_value}mmHg → 상태: {sbp_status}")
            print(f"   (대한고혈압학회 기준: 정상 <120, 주의 120-129, 고혈압 1기 130-139, 고혈압 2기 ≥140)")
        # 이완기 혈압 관련 콘솔 출력 부분은 제거
        print(f"   {disease_name}과 가장 높게 연관된 {primary_feature_label}은(는) 높아지면 고혈압 위험이 증가합니다.\n") # secondary_feature_label 제거

    elif disease_name == "당뇨병":
        if primary_user_value is not None:
            glu_status = ""
            if primary_user_value <= 99: glu_status = "정상"
            elif 100 <= primary_user_value <= 125: glu_status = "주의"
            else: glu_status = "고위험"
            normal_range_glu = "70~99mg/dL"
            warning_range_glu = "100~125mg/dL"
            high_risk_range_glu = "≥126mg/dL"
            print(f"3. {primary_feature_label} 수치: {primary_user_value}mg/dL → 상태: {glu_status}")
            print(f"   (정상: {normal_range_glu}, 주의: {warning_range_glu}, 고위험: {high_risk_range_glu})")
        # 당화혈색소 관련 콘솔 출력 부분은 제거
        print(f"   {disease_name}과 가장 높게 연관된 {primary_feature_label}은(는) 높아지면 {disease_name} 위험이 증가합니다.\n") # secondary_feature_label 제거


    elif disease_name == "비만":
        if primary_user_value is not None:
            bmi_status = ""
            if primary_user_value < 18.5: bmi_status = "저체중"
            elif 18.5 <= primary_user_value <= 22.9: bmi_status = "정상 체중"
            elif 23.0 <= primary_user_value <= 24.9: bmi_status = "과체중"
            elif 25.0 <= primary_user_value <= 29.9: bmi_status = "비만 1단계"
            elif 30.0 <= primary_user_value <= 34.9: bmi_status = "비만 2단계"
            else: bmi_status = "초고도비만"
            normal_range_bmi = "18.5~22.9"
            warning_range_bmi = "23.0~24.9 이상"
            print(f"3. {primary_feature_label} 수치: {primary_user_value} → 상태: {bmi_status}")
            print(f"   (정상: {normal_range_bmi}, 과체중: {warning_range_bmi})")
        # 허리둘레 관련 콘솔 출력 부분은 제거
        print(f"   {disease_name}과 가장 높게 연관된 {primary_feature_label}은(는) 높아지면 {disease_name} 위험이 증가합니다.\n") # secondary_feature_label 제거


    return health_score, result['class_name']

# 대표 feature 맵 (주요 지표만 사용하도록 업데이트)
top_features_map = {
    "당뇨병": {"primary": "HE_glu"}, # 당화혈색소 제거
    "고혈압": {"primary": "HE_sbp1"}, # 이완기 혈압 제거
    "비만": {"primary": "HE_BMI"} # 허리둘레 제거
}

# 질병별 결과를 묶기
disease_results = {
    "당뇨병": result_dm,
    "고혈압": result_hp,
    "비만": result_obesity
}

health_scores = []
disease_risk_statuses = {}

for disease_name, result in disease_results.items():
    primary_feature = top_features_map[disease_name]["primary"]
    # secondary_feature는 더 이상 사용하지 않으므로 제거
    
    current_validated_user_data = None
    if disease_name == "당뇨병":
        current_validated_user_data = user_data_dm
    elif disease_name == "고혈압":
        current_validated_user_data = user_data_hp
    elif disease_name == "비만":
        current_validated_user_data = user_data_obesity
    
    user_sex_val = current_validated_user_data.get('sex')

    primary_user_value = current_validated_user_data.get(primary_feature)

    if primary_user_value is not None:
        # generate_disease_report 함수 호출 시 secondary_feature_name, secondary_user_value는 None으로 전달
        score, risk_status = generate_disease_report(
            user_name="홍길동",
            disease_name=disease_name,
            result=result,
            primary_feature_name=primary_feature,
            primary_user_value=primary_user_value,
            secondary_feature_name=None, # 명시적으로 None 전달
            secondary_user_value=None,    # 명시적으로 None 전달
            user_sex=user_sex_val
        )
        health_scores.append(score)
        disease_risk_statuses[disease_name] = risk_status
    else:
        print(f"{disease_name}에 대한 {primary_feature} 값이 user_data에 없습니다.")

# 총점 출력 (기존 코드 유지)
if health_scores:
    total_score = sum(health_scores) / len(health_scores)
    print("---")
    print(f"### 총 개인 건강 점수: {total_score:.1f}점")
    print("(당뇨병, 고혈압, 비만에 대한 정상상태 점수를 평균한 값입니다.)\n")

# summary_list_of_lists 생성 부분 대폭 수정
summary_list_of_lists = []

for disease_name, result in disease_results.items():
    primary_feature_code = top_features_map[disease_name]["primary"]

    current_validated_user_data = None
    if disease_name == "당뇨병":
        current_validated_user_data = user_data_dm
    elif disease_name == "고혈압":
        current_validated_user_data = user_data_hp
    elif disease_name == "비만":
        current_validated_user_data = user_data_obesity
    
    user_value_primary = current_validated_user_data.get(primary_feature_code)

    # 주요 지표 정보만 분리하여 리스트에 추가
    feature_value_str = ""
    status_str = ""
    normal_range_str = ""

    # 고혈압
    if disease_name == "고혈압":
        sbp_status = ""
        if user_value_primary is not None:
            if user_value_primary < 120: sbp_status = "정상"
            elif 120 <= user_value_primary <= 129: sbp_status = "주의"
            elif 130 <= user_value_primary <= 139: sbp_status = "위험"
            else: sbp_status = "고위험"
        
        feature_value_str = f"수축기 혈압({user_value_primary}mmHg)"
        status_str = sbp_status
        normal_range_str = "정상 기준: <120mmHg"

    # 당뇨병
    elif disease_name == "당뇨병":
        glu_status = ""
        if user_value_primary is not None:
            if user_value_primary <= 99: glu_status = "정상"
            elif 100 <= user_value_primary <= 125: glu_status = "주의"
            else: glu_status = "고위험"
        
        feature_value_str = f"공복혈당({user_value_primary}mg/dL)"
        status_str = glu_status
        normal_range_str = "정상 기준: 70~99mg/dL"
    
    # 비만
    elif disease_name == "비만":
        bmi_status = ""
        if user_value_primary is not None:
            if user_value_primary < 18.5: bmi_status = "저체중"
            elif 18.5 <= user_value_primary <= 22.9: bmi_status = "정상 체중"
            elif 23.0 <= user_value_primary <= 24.9: bmi_status = "과체중"
            elif 25.0 <= user_value_primary <= 29.9: bmi_status = "비만 1단계"
            elif 30.0 <= user_value_primary <= 34.9: bmi_status = "비만 2단계"
            else: bmi_status = "초고도비만"
        
        feature_value_str = f"BMI({user_value_primary})"
        status_str = bmi_status
        normal_range_str = "정상 기준: 18.5~22.9"
    
    # 최종 리스트 요소 구성 (순서 조정 및 중복 제거)
    summary_item = [
        disease_name,                 # 1. 질병 이름
        result["class_name"],         # 2. 예측된 질병 상태 (예: 고혈압 전단계)
        f"{float(result['probabilities'][result['predicted_class']-1]) * 100:.0f}%", # 3. 예측된 상태의 확률
        f"{model_scores.get(disease_name, 0.0):.1f}%", # 4. 모델 정확도
        feature_value_str,            # 5. 주요 지표 값 (예: "공복혈당(100.0mg/dL)")
        status_str,                   # 6. 주요 지표 상태 (예: "주의")
        normal_range_str              # 7. 주요 지표 정상 기준 (예: "정상 기준: 70~99mg/dL")
    ]

    summary_list_of_lists.append(summary_item)

print(json.dumps(summary_list_of_lists, indent=2, ensure_ascii=False))

모델 정확도 로드 성공.
=== 당뇨병 예측 결과 ===
1. 홍길동님의 현재 당뇨병 관련 상태는 당뇨병 입니다.
   (정상 확률: 0%, 경계형 당뇨 확률: 46%, 당뇨병 확률: 54%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. 공복혈당 수치: 100.0mg/dL → 상태: 주의
   (정상: 70~99mg/dL, 주의: 100~125mg/dL, 고위험: ≥126mg/dL)
   당뇨병과 가장 높게 연관된 공복혈당은(는) 높아지면 당뇨병 위험이 증가합니다.

=== 고혈압 예측 결과 ===
1. 홍길동님의 현재 고혈압 관련 상태는 고혈압 전단계 입니다.
   (정상 혈압 확률: 0%, 주의 혈압 단계 확률: 40%, 고혈압 전단계 확률: 59%, 고혈압 확률: 2%)

2. 건강 점수: 0.0점 (이 점수는 정상 확률 기준입니다.)

3. 수축기 혈압 수치: 135mmHg → 상태: 위험
   (대한고혈압학회 기준: 정상 <120, 주의 120-129, 고혈압 1기 130-139, 고혈압 2기 ≥140)
   고혈압과 가장 높게 연관된 수축기 혈압은(는) 높아지면 고혈압 위험이 증가합니다.

=== 비만 예측 결과 ===
1. 홍길동님의 현재 비만 관련 상태는 정상 체중 입니다.
   (저체중 확률: 0%, 정상 체중 확률: 100%, 과체중 확률: 0%, 비만 확률: 0%, 고도 비만 확률: 0%, 초고도 비만 확률: 0%)

2. 건강 점수: 100.0점 (이 점수는 정상 확률 기준입니다.)

3. BMI 수치: 21.8 → 상태: 정상 체중
   (정상: 18.5~22.9, 과체중: 23.0~24.9 이상)
   비만과 가장 높게 연관된 BMI은(는) 높아지면 비만 위험이 증가합니다.

---
### 총 개인 건강 점수: 33.3점
(당뇨병, 고혈압, 비만에 대한 정상상태 점수를 평균한 값입니다.)

[
  [
    "당뇨병",
    "당뇨병",
    "54%",
    "94.0%",
    "공복혈당(100.

**딕셔너리에 key와 값 저장 or list에 값만 저장**

In [ ]:
# # ✅ 핵심 정보 요약 리스트
# summary_keys = []

# # ✅ 주요 지표 이름 매핑
# feature_name_map = {
#     "HE_glu": "공복혈당",
#     "HE_sbp1": "수축기 혈압",
#     "HE_BMI": "BMI"
# }

# for disease_name, result in disease_results.items():
#     top_feature_code = top_features[disease_name]
#     top_feature_name = feature_name_map.get(top_feature_code, top_feature_code)
#     user_value = user_data.get(top_feature_code, None)

#     # 지표 상태 및 기준 설정
#     if top_feature_code == "HE_sbp1":
#         if user_value <= 120:
#             status = "정상"
#         elif 121 <= user_value <= 139:
#             status = "주의"
#         else:
#             status = "고위험"
#         normal_range = "90~120"

#     elif top_feature_code == "HE_glu":
#         if user_value <= 99:
#             status = "정상"
#         elif 100 <= user_value <= 125:
#             status = "주의"
#         else:
#             status = "고위험"
#         normal_range = "70~99"

#     elif top_feature_code == "HE_BMI":
#         if user_value < 18.5:
#             status = "저체중"
#         elif 18.5 <= user_value <= 22.9:
#             status = "정상 체중"
#         elif 23.0 <= user_value <= 24.9:
#             status = "과체중"
#         elif 25.0 <= user_value <= 29.9:
#             status = "비만 1단계"
#         elif 30.0 <= user_value <= 34.9:
#             status = "비만 2단계"
#         else:
#             status = "비만 3단계"
#         normal_range = "18.5~22.9"
#     else:
#         status = "알 수 없음"
#         normal_range = "-"

#     # ✅ 예측된 클래스의 확률만 추출
#     predicted_index = result['predicted_class'] - 1
#     predicted_prob = f"{float(result['probabilities'][predicted_index]) * 100:.0f}%"

#     # ✅ 사용자 수치 + 정상 기준 합치기
#     value_with_range = (
#         f"{str(int(user_value)) if user_value == int(user_value) else str(round(user_value, 1))} "
#         f"(정상 기준: {normal_range})"
#     )
#     from sklearn.metrics import classification_report

#     # 테스트 결과 리포트 (딕셔너리 형태)
#     report_dm = classification_report(y_test_dm, y_pred_dm, output_dict=True)
#     report_hp = classification_report(y_test_hp, y_pred_hp, output_dict=True)
#     report_obe = classification_report(y_test_obe, y_pred_obe, output_dict=True)

#     # ✅ 정확도 (accuracy)만 추출
#     model_scores = {
#      "당뇨병": round(report_dm["accuracy"] * 100, 1),
#         "고혈압": round(report_hp["accuracy"] * 100, 1),
#      "비만": round(report_obe["accuracy"] * 100, 1)
#     }
    
#     #딕셔너리로하면 값들만 넣지 못해서 우선 리스트로 해놨습니다. 필요하면 아래 딕셔너리 쓰시면 될 것 같아요.
#     #✅ 핵심 요약만 담은 summary (리스트)
#     summary = [
#     disease_name,                      # 질병명
#     result["class_name"],             # 예측 결과
#     predicted_prob,                   # 예측 확률
#     f"{model_scores[disease_name]}%", # 모델 정확도
#         [                                 # 주요지표 정보도 리스트로
#         top_feature_name,             # 주요지표 이름
#         status,                       # 사용자 상태
#         value_with_range              # 수치 (정상 기준 포함)
#         ]
#     ]

#     # # ✅ 핵심 요약만 담은 summary (딕셔너리)
#     # summary = {
#     #       "질병명": disease_name,
#     #      "예측결과": result["class_name"],
#     #      "예측확률": predicted_prob,
#     #       "모델정확도": f"{model_scores[disease_name]}%",
#     #       "주요지표": {
#     #         "지표이름": top_feature_name,
#     #         "판정": status,
#     #         "수치정보": value_with_range
#     #     }
#     # }


#     summary_keys.append(summary)
# # ✅ 최종 결과 출력
# import json
# print(json.dumps(summary_keys, indent=2, ensure_ascii=False))


[
  [
    "당뇨병",
    "정상",
    "81%",
    "75.8%",
    [
      "공복혈당",
      "정상",
      "90 (정상 기준: 70~99)"
    ]
  ],
  [
    "고혈압",
    "고혈압",
    "72%",
    "64.6%",
    [
      "수축기 혈압",
      "고위험",
      "150 (정상 기준: 90~120)"
    ]
  ],
  [
    "비만",
    "정상 체중",
    "98%",
    "91.5%",
    [
      "BMI",
      "정상 체중",
      "21.8 (정상 기준: 18.5~22.9)"
    ]
  ]
]
